In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from lightning.pytorch.callbacks import EarlyStopping, ModelSummary, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
import torch

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

In [5]:
# fig = plt.figure(figsize=(25, 25))
# sns.heatmap(train_df.corr(), yticklabels=False, cbar=False, cmap="viridis", vmax=0.8)

In [6]:
# fig = plt.figure(figsize=(20, 20))
# sns.heatmap(train_df.isnull(), yticklabels=False, cbar=False, cmap="viridis")

In [7]:
train_df.corr()["SalePrice"].sort_values()

ExterQual_TA       -0.589044
KitchenQual_TA     -0.519298
BsmtQual_TA        -0.452394
GarageFinish_Unf   -0.410608
MasVnrType_None    -0.374468
                      ...   
GarageArea          0.623431
GarageCars          0.640409
GrLivArea           0.708624
OverallQual         0.790982
SalePrice           1.000000
Name: SalePrice, Length: 290, dtype: float64

In [8]:
train_df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [9]:
train_df.fillna(train_df.median())
test_df.fillna(test_df.median())

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0,0,0,1,0,0,0,0,1,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0,0,0,1,0,0,0,0,1,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0,0,0,1,0,0,0,0,1,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0,0,0,1,0,0,0,0,1,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,0,0,0,1,1,0,0,0,0,0
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,0,0,0,1,1,0,0,0,0,0
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,0,0,0,1,0,0,0,0,1,0


In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [11]:
att_train = train_df.columns
att_test = test_df.columns
not_included = att_train ^ att_test
Y_train = train_df["SalePrice"]
train_df.drop(not_included, axis=1, inplace=True)
# test_df.drop(not_included, axis=1, inplace=True)
X_train = train_df.drop("Id", axis=1)
X_test = test_df.drop("Id", axis=1)

C:\Users\tomkj\AppData\Local\Temp\ipykernel_27448\2449702125.py:3: FutureWarning: Index.__xor__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__xor__.  Use index.symmetric_difference(other) instead.
  not_included = att_train ^ att_test


In [12]:
X_train.isna().sum()

MSSubClass                 0
LotFrontage              259
LotArea                    0
OverallQual                0
OverallCond                0
                        ... 
SaleCondition_AdjLand      0
SaleCondition_Alloca       0
SaleCondition_Family       0
SaleCondition_Normal       0
SaleCondition_Partial      0
Length: 270, dtype: int64

In [13]:
X_train = X_train.fillna(X_train.median())
X_test = X_test.fillna(X_test.median())
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [14]:
import lightning as L
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

In [23]:
class HouseModel(L.LightningModule):
    def __init__(self, scaler):
        super().__init__()

        self.example_input_array = torch.Tensor(5, 270)
        
        self.net = nn.Sequential(
            nn.Linear(270, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
        self.criterion = nn.HuberLoss()
        self.scaler = scaler
    
    def forward(self, x):
        x = self.net(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.net(x)
        loss = self.criterion(y_pred, y)
        abs_error = abs(y_pred.detach().numpy() - y.detach().numpy())
        print(abs_error)
        abs_error = self.scaler.inverse_transform(abs_error)

        self.log("loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        self.log("abs_error", abs_error, prog_bar=True, on_step=True, on_epoch=True)

        return {"loss", loss}
    
    def configure_optimizers(self):
        return optim.Adam(self.net.parameters(), lr=1e-3)

In [24]:


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)

train_ds = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(dataset=train_ds, shuffle=True, batch_size=32)
model = HouseModel(scaler)
summary = ModelSummary(max_depth=-1)
early_stopping = EarlyStopping(monitor="train_loss", mode="min", patience=10)
tb_logger = TensorBoardLogger("results", name="my_model")

In [25]:
trainer = L.Trainer(max_epochs=100, callbacks=[early_stopping, summary], logger=tb_logger,gpus=0, fast_dev_run=True)


c:\Users\tomkj\miniconda3\envs\ai\lib\site-packages\lightning\pytorch\trainer\connectors\accelerator_connector.py:474: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [26]:
trainer.fit(model=model, train_dataloaders=train_dataloader)



  | Name      | Type       | Params | In sizes | Out sizes
----------------------------------------------------------------
0 | net       | Sequential | 21.6 K | [5, 270] | [5, 1]   
1 | net.0     | Linear     | 17.3 K | [5, 270] | [5, 64]  
2 | net.1     | ReLU       | 0      | [5, 64]  | [5, 64]  
3 | net.2     | Linear     | 4.2 K  | [5, 64]  | [5, 64]  
4 | net.3     | ReLU       | 0      | [5, 64]  | [5, 64]  
5 | net.4     | Linear     | 65     | [5, 64]  | [5, 1]   
6 | net.5     | Sigmoid    | 0      | [5, 1]   | [5, 1]   
7 | criterion | HuberLoss  | 0      | ?        | ?        
----------------------------------------------------------------
21.6 K    Trainable params
0         Non-trainable params
21.6 K    Total params
0.086     Total estimated model params size (MB)
c:\Users\tomkj\miniconda3\envs\ai\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a

Training: 0it [00:00, ?it/s]

c:\Users\tomkj\miniconda3\envs\ai\lib\site-packages\torch\nn\modules\loss.py:988: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)


AAAAA


ValueError: operands could not be broadcast together with shapes (32,32) (270,) (32,32) 